<h2><center> <font color='cyan'> Urban Heat Scan Workflow </font> </center></h2>
<h3><center> <font color='cyan'>Table of Contents</font>   </center></h3>

[Preface: Import dependencies and set paths](#section0)
<br>
[Section 1: Big picture: City map and context](#section1)
<br>
[Section 2: Population density and tree cover city level](#section2) 
<br>
[Section 3: Visualize Vito heat rasters at city level](#section3) 
<br>
[Section 4: Create the Heat Scan PPT](#section4) 
<br>
[Section 5: Population density and tree cover Sub-city level. NA ](#section5) 




<a id='section0'></a>
<h5><center> <font color='cyan'> Preface: Import dependencies and set paths</font>   </center></h5>

In [1]:
# Import packages
import geopandas as gpd , os , time , math, glob
from collections import defaultdict

from urllib.request import urlopen
from pyproj import CRS
from os.path import exists
from geopandas.tools import overlay
from rasterio.warp import calculate_default_transform, reproject, Resampling
from osgeo import ogr
import networkx as nx
import osmnx as ox
ox.config(use_cache=True, log_console=True)

import  seaborn as sn
import geemap 
import ee 
import rasterio
from rasterio.mask import mask
from rasterio.features import shapes
import osmnx as ox
from shapely.geometry import box
from rasterio.plot import show
import pylab as plt
from geopandas.tools import sjoin
import contextily as cx
from h3 import h3
import h3pandas
from rasterstats import zonal_stats
import pandas as pd
import numpy as np
from rasterio.crs import CRS
import matplotlib.colors as colors
from matplotlib.lines import Line2D
from IPython.core.interactiveshell import InteractiveShell
import warnings
warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "last_expr" #Supress matplotlib output

# Stats heavy-lifting
from esda.moran import Moran
from libpysal.weights import Queen, KNN
import  libpysal

from sklearn import metrics
from sklearn.preprocessing import robust_scale
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from pysal.lib import weights
from pysal.explore import esda
from pysal.model import spreg

from pptx import Presentation
from pptx.util import Inches
from pptx.util import Inches, Pt

from pptx.oxml.xmlchemy import OxmlElement
from pptx.util import Cm
from pptx.enum.shapes import MSO_SHAPE
from pptx.dml.color import RGBColor
import plotly.graph_objects as go

plt.rcParams['hatch.color'] = 'white'
plt.rcParams['hatch.linewidth'] = 4.2
plt.rcParams['font.family'] = 'Yu Gothic'

# plt.rcParams['figure.figsize'] = 7.5 , 7.5
width=10
height=7.6
plt.rcParams['figure.figsize'] = width , height

C:\Users\Aziz\AppData\Local\Temp\ipykernel_3576\3909042368.py:13: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


In [2]:
def set_paths(base_dir):
    data = os.path.join(base_dir, 'data') 
    output = os.path.join(base_dir, 'output') 
    shapefiles = os.path.join(output, 'shapefiles') 
    maps = os.path.join(output, 'maps') 
    rasters = os.path.join(output, 'rasters') 
    tables = os.path.join(output, 'tables') 
    
    dirs_list= [data, output, base_dir, shapefiles , maps , rasters , tables]
    for dir in dirs_list:
        if not os.path.exists(dir):
            os.mkdir(dir)

    return data, shapefiles , maps , rasters , output ,tables



In [3]:

# Initialize ee and import shapefile and select city
ee.Initialize()
run_vito_analysis=False #True
run_tree_and_pop_analysis=False #True
create_the_ppt=True
WGS84=4326 #uses degrees
WGS84_meters=3857 #uses meters
EPSG_str= 'EPSG:4326'
crs = 4326

city= "Nis City"

base_dir = f'C:/Users/Aziz/Dropbox/CRP/UHI/{city}'
data, shapefiles , maps , rasters , output ,tables = set_paths(base_dir)

# shapefile_path=f'{shapefiles}/geoBoundaries-SRB-ADM1_simplified.shp'

# Sub city donot exist somehow. City and municipailty are the same
# Serbia is divided into 145 municipalities and 29 cities,[2] which form the basic units of local government. 
# Each municipality has its own assembly (elected every four years in local elections), a municipal president, 
# public service property and a budget. Municipalities usually have more than 10,000 inhabitants.[2]

shapefile_path=f'{shapefiles}/geoBoundaries-SRB-ADM2.shp'
gdf_city = gpd.read_file(shapefile_path).to_crs(WGS84)
gdf_city=gdf_city[gdf_city["shapeName"] == city]
country= gdf_city.iloc[0]['shapeGroup']


<a id='section1'></a>
<h5><center> <font color='cyan'> Section 1: Big picture: City map</font>   </center></h5>


In [4]:
%%capture
def map_city(maps, vector_file, crs, legend_title,  visualize_column, title, map_output):
    ax = vector_file.to_crs(crs).plot(figsize=(10, 10), \
                                   column= visualize_column, \
                                   alpha=0.6,  \
                                   facecolor='none',edgecolor='yellow' , \
                                   linewidth=3 , 
                                   legend=True,  
                                   legend_kwds={'loc':'upper right', 
                                                'bbox_to_anchor':(1, 1), 
                                                'markerscale':1.01, 
                                                'title_fontsize':'small', 
                                                'fontsize':'x-small'
                                                }  
                                        )

    ax.set_aspect('equal')
    cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery,  crs=crs) 
    cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels,   crs=crs, zoom=11) # zoom=13

    vector_file_buffer = vector_file.to_crs(crs).buffer(0.016) #for zoom out
    minx, miny, maxx, maxy = vector_file_buffer.total_bounds
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)
    # Legends
    LegendElement = [
                    Line2D([0],[0],color='yellow',lw=3,label=f'{legend_title}')
                    ]
    ax.legend(handles=LegendElement,loc='upper right')
    plt.xticks([])
    plt.yticks([])   
    ax.title.set_text(f'{title}')
    plt.tight_layout()
    ax.figure.savefig(map_output,  bbox_inches='tight',   dpi = 400)

<a id='section2'></a>
<h5><center> <font color='cyan'> Section 2: Population density and tree cover city level</font>   </center></h5>

In [5]:
def create_fc_collection(gdf_city):
    poly = gdf_city.geometry.unary_union
    gdf_boundary = gpd.GeoDataFrame(geometry=[poly],crs=gdf_city.crs)
    geom = gdf_boundary['geometry']
    jsonDict = eval(geom.to_json())
    for index, row in gdf_boundary.iterrows(): 
        polygon_list= []
        for x in jsonDict['features'][index]['geometry']['coordinates']:
            polygon_list.append(x)
            region = ee.Geometry.Polygon(polygon_list)
            fc_filtered = ee.FeatureCollection(region)
    # gdf_boundary.plot()
    return fc_filtered  , region 


def clipToCol(image):
  """clip gee collection
  args:
      image: Image collection
  returns:
    ee-collection: Clipped image collection
   
   """
  return image.clip(fc_filtered)

def download_tree_cover(ee , EPSG_str, region):
    tree_cover=ee.ImageCollection("projects/sat-io/open-datasets/GFCC30TC") \
        .limit(1, 'system:time_start', False).first().clip(fc_filtered)
    tree_cover_tif = os.path.join(rasters, 'tree_cover_projected.tif')
    geemap.ee_export_image(
        tree_cover, filename=tree_cover_tif,  
        crs=EPSG_str,
        # crs_transform=crs_transform,
        scale=30, region=region, file_per_band=False
    )
    return tree_cover_tif

def download_tree_cover_esa_vito(ee , EPSG_str, region):
    tree_cover=ee.ImageCollection("ESA/WorldCover/v100") \
        .limit(1, 'system:time_start', False).first().clip(fc_filtered).eq(10) #trees only
        # .limit(1, 'system:time_start', False).select(10).first().clip(fc_filtered) #trees only
    tree_cover_tif_esa = os.path.join(rasters, 'tree_cover_projected_esa.tif')
    geemap.ee_export_image(
        tree_cover, filename=tree_cover_tif_esa,  
        crs=EPSG_str,
        # crs_transform=crs_transform,
        scale=10, region=region, file_per_band=False
    )
    return tree_cover_tif_esa


def dowload_pop_density(ee , EPSG_str, region):
    #  Define WorldPop & clip using function 
    pop_density = ee.ImageCollection("WorldPop/GP/100m/pop").map(clipToCol). \
                    filterDate('2020').select('population').mosaic()

    pop_density_tif = os.path.join(rasters, 'pop_density_projected.tif')
    geemap.ee_export_image( pop_density, 
                           filename=pop_density_tif,  
                           crs=EPSG_str,
                           # crs_transform=crs_transform,
                           scale=100, 
                           region=region, 
                           file_per_band=False
                           )

    return pop_density_tif

# Takes longer
def download_google_dynamic_worldcover(ee , EPSG_str, region):
    # dynamic_world_cover=ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1") \
    #     .limit(1, 'system:time_start', False).map(clipToCol).mosaic() #all only
    dynamic_world_cover=ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1") \
        .limit(1, 'system:time_start', False).mosaic().clip(fc_filtered)
    google_dynamic_world_cover_projected = os.path.join(rasters, 'google_dynamic_world_cover_projected.tif')
    geemap.ee_export_image(
        dynamic_world_cover, filename=google_dynamic_world_cover_projected,  
        crs=EPSG_str,
        # crs_transform=crs_transform,
        scale=10, region=region, file_per_band=False
    )
    return google_dynamic_world_cover_projected

def download_esri_dynamic_worldcover(ee , EPSG_str, region): 
    dynamic_world_cover=ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS")\
        .filterDate('2022-01-01','2022-12-31').mosaic().clip(fc_filtered)
    esri_dynamic_world_cover_projected = os.path.join(rasters, 'esri_dynamic_world_cover_projected.tif')
    geemap.ee_export_image(
        dynamic_world_cover, filename=esri_dynamic_world_cover_projected,  
        crs=EPSG_str,
        # crs_transform=crs_transform,
        scale=10, region=region, file_per_band=False
    )
    return esri_dynamic_world_cover_projected


def reproject_rasters(crs ,output_dir, unprojected_raster, projected_raster):
    if not exists(projected_raster):
        with rasterio.open(unprojected_raster) as src:
            dst_crs = 'EPSG:' + str(crs)
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
                # ,'dtype': np.float32
            })

            with rasterio.open(projected_raster, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        # dtype= np.float32,
                        resampling=Resampling.nearest)

def polygonize(raster_file ,output_shapefile, crs, mask_value, raster_val):
    # mask = None
    with rasterio.Env():
        with rasterio.open(raster_file ) as src: #, dtype= np.float32
            image = src.read(1) # first band
            no_data_value=0
            image = np.where(image<0,no_data_value,image) #replaced negative values with 0
            results = (
            {'properties': {raster_val : v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                shapes(image, mask=mask_value, transform=src.transform)))

    # The result is a generator of GeoJSON features
    geoms = list(results)
    # first feature
    # print(geoms[0])
    gpd_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)
    crs_utm = CRS.from_user_input(crs)
    driver= 'ESRI Shapefile'
    gpd_polygonized_raster.to_file(output_shapefile, driver, crs=crs_utm)
    return gpd_polygonized_raster


def calcBearing (lat1, long1, lat2, long2):
    dLon = (long2 - long1)
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
    bearing = math.atan2(x,y)   # use atan2 to determine the quadrant
    bearing = math.degrees(bearing)
    return bearing

def calcNSEW(lat1, long1, lat2, long2):
    points = ["north", "north east", "east", "south east", "south", "south west", "west", "north west"]
    bearing = calcBearing(lat1, long1, lat2, long2)
    bearing += 22.5
    bearing = bearing % 360
    bearing = int(bearing / 45) # values 0 to 7
    NSEW = points [bearing]
    return NSEW

def ordinal(n: int):
    if 11 <= (n % 100) <= 13:
        suffix = 'th'
    else:
        suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
    return str(n) + suffix

def get_osm_tags_inside_clusters(cluster_boundaries):   
    for index, poi in cluster_boundaries.iterrows():
        polygon = cluster_boundaries.iloc[index]['geometry']
        park , education, industry, building=get_services_infrastructure_inside_poly(polygon)
    return park , education, industry, building

def delete_shapefile(output_shapefile):
    shapefile = ogr.Open(output_shapefile,1 )
    layer=shapefile.GetLayerByIndex(0)
    count=layer.GetFeatureCount()
    for feature in range(count):
        layer.DeleteFeature(feature)


def map_clusters(maps, hexagons, legend_title,legends_format, crs ,cmap,  visualize_column, title):
    hexagons=hexagons.to_crs(crs)
    hexagons = hexagons.sort_values('cluster_area_deg', ascending = False).groupby('ward5wknn').head(2)
    hexagons=hexagons.reset_index()
    for index, poi in hexagons.iterrows():
        try:
            cluster_class=poi["ward5wknn"]
            polygon = hexagons[hexagons["ward5wknn"]==index]
            # polygon = hexagons.iloc[index]['geometry']  
            map_output= f'{maps}/{city}_{visualize_column}_{index}_{cluster_class}_cluster.png'
            ax = polygon.plot(figsize=(10, 10), \
                                        column= visualize_column, \
                                        facecolor='none',edgecolor='cyan' , \
                                        linewidth=1.5 , \
                                        alpha=0.6, categorical=True , cmap=cmap, \
                                        legend=True,  # Add legend 
                                        legend_kwds={'loc':'upper right', 
                                                        'bbox_to_anchor':(1, 1), 
                                                        'fmt':legends_format,
                                                        'markerscale':1.01, 
                                                        'title_fontsize':'small', 
                                                        'fontsize':'x-small'
                                                        # title_fontsizeint or {'xx-small', 'x-small', 'small'
                                                        } ,
                                        aspect=1
                                                )
            # vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
            polygon_buffer = polygon.to_crs(crs).buffer(0.016) #for zoom out
            minx, miny, maxx, maxy = polygon_buffer.total_bounds
            ax.set_xlim(minx, maxx)
            ax.set_ylim(miny, maxy)

            cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery,  crs=crs) 
            # vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
            # cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite,  crs=crs , zoom=12) 
            cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels,   crs=crs, zoom=11) # zoom=13

            # plt.title(f'{title}',fontsize=16)
            plt.xticks([])
            plt.yticks([]) 
            leg1 = ax.get_legend()
            # Set markers to square shape
            for ea in leg1.legendHandles:
                ea.set_marker('s')
            leg1.set_title(f'{legend_title}')
            ax.title.set_text(f'{title}')
            plt.tight_layout()
            # ax.figure.savefig(map_output)
            ax.figure.savefig(map_output,  bbox_inches='tight' ,   dpi = 400)

        except:
            pass
        
def map_clusters_land_cover(maps, hexagons, legend_title,legends_format, crs ,cmap,  visualize_column, title):
    hexagons=hexagons.to_crs(crs)
    hexagons=hexagons.reset_index()
    hexagons = hexagons.sort_values('cluster_area_deg', ascending = False).groupby('ward5wknn').head(2)
    for index, poi in hexagons.iterrows():
        try:
            cluster_class=poi["ward5wknn"]
            polygon = hexagons[hexagons["ward5wknn"]==index]
            # polygon = hexagons.iloc[index]['geometry']  
            map_output= f'{maps}/{city}_{visualize_column}_{index}_{cluster_class}_cluster.png'
            ax = polygon.plot(figsize=(10, 10), \
                                        column= visualize_column, \
                                        facecolor='none',edgecolor='cyan' , \
                                        linewidth=1.5 , \
                                        alpha=0.6, categorical=True , cmap=cmap, \
                                        legend=True,  # Add legend 
                                        legend_kwds={'loc':'upper right', 
                                                        'bbox_to_anchor':(1, 1), 
                                                        'fmt':legends_format,
                                                        'markerscale':1.01, 
                                                        'title_fontsize':'small', 
                                                        'fontsize':'x-small'
                                                        # title_fontsizeint or {'xx-small', 'x-small', 'small'
                                                        } ,
                                        aspect=1
                                                )
            # vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
            polygon_buffer = polygon.to_crs(crs).buffer(0.016) #for zoom out
            minx, miny, maxx, maxy = polygon_buffer.total_bounds
            print("Pre-non bounds", minx, miny, maxx, maxy)
            if not [x for x in (minx, miny, maxx, maxy) if x is None]:
                print(minx, miny, maxx, maxy)
                ax.set_xlim(minx, maxx)
                ax.set_ylim(miny, maxy)

                cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery,  crs=crs) 
                # vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
                # cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite,  crs=crs , zoom=12) 
                cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels,   crs=crs, zoom=11) # zoom=13

                plt.title(f'{title}',fontsize=16)
                plt.xticks([])
                plt.yticks([]) 
                leg1 = ax.get_legend()
                # Set markers to square shape
                for ea in leg1.legendHandles:
                    ea.set_marker('s')
                leg1.set_title(f'{legend_title}')
                ax.title.set_text(f'{title}')
                plt.tight_layout()
                # ax.figure.savefig(map_output)
                ax.figure.savefig(map_output,  bbox_inches='tight' ,   dpi = 400)
        except:
            pass


def map_func(maps, vector_file,  hexagons, legend_title,legends_format, crs ,cmap,  visualize_column, title, map_output, scheme, categorical):
    if categorical==True:
        ax = hexagons.to_crs(crs).plot(figsize=(10, 10), \
                                    column= visualize_column, \
                                    alpha=0.6, categorical=True , cmap=cmap, \
                                    legend=True,  # Add legend 
                                    legend_kwds={'loc':'upper right', 
                                                    'bbox_to_anchor':(1, 1), 
                                                    'fmt':legends_format,
                                                    'markerscale':1.01, 
                                                    'title_fontsize':'small', 
                                                    'fontsize':'x-small'
                                                    # title_fontsizeint or {'xx-small', 'x-small', 'small'
                                                    } ,
                                    aspect=1
                                            )
    elif categorical==False:
        ax = hexagons.to_crs(crs).plot(figsize=(10, 10), \
                                    column= visualize_column, \
                                    alpha=0.6, scheme=scheme, cmap=cmap, \
                                    legend=True,  # Add legend 
                                    legend_kwds={'loc':'upper right', 
                                                    'bbox_to_anchor':(1, 1), 
                                                    'fmt':legends_format,
                                                    'markerscale':1.01, 
                                                    'title_fontsize':'small', 
                                                    'fontsize':'x-small'
                                                    # title_fontsizeint or {'xx-small', 'x-small', 'small'
                                                    } ,
                                    aspect=1
                                            )
    # vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
    # cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery,  crs=crs) 
    vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
    cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite,  crs=crs , zoom=12) 
    cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels,   crs=crs, zoom=11) # zoom=13

    # plt.title(f'{title}',fontsize=16)
    plt.xticks([])
    plt.yticks([]) 
    leg1 = ax.get_legend()
    # Set markers to square shape
    for ea in leg1.legendHandles:
        ea.set_marker('s')
    leg1.set_title(f'{legend_title}')
    ax.title.set_text(f'{title}')
    plt.tight_layout()
    # ax.figure.savefig(map_output)
    ax.figure.savefig(map_output,  bbox_inches='tight' ,   dpi = 400)


def map_land_cover(vector_file,  hexagons,title, legend_title, crs,  visualize_column, map_output):
    class_labels = {1: "Water , blue", 2: "Trees, green", 4:"Flooded Vegetation , orange", 5:"Crops, seagreen", 7:"Built Area, tan", 8:"Bare Ground, goldenrod", 9:"Snow/Ice, snow", 10:"Clouds, ghostwhite", 11:"Rangeland, greenyellow" }
    hexagons[['class', 'color']] = hexagons['esri_dyna'].map(class_labels).str.split(', ',expand=True).values
    color_map= dict(zip(hexagons["class"], hexagons["color"]))
    ax = hexagons.to_crs(crs).plot(figsize=(10, 10), \
                                column= visualize_column, \
                                alpha=0.6, categorical=True , color=hexagons.color, \
                                legend=True,  # Add legend 
                                legend_kwds={'loc':'upper right', 
                                                'bbox_to_anchor':(1, 1), 
                                                'fmt':legends_format,
                                                'markerscale':1.01, 
                                                'title_fontsize':'small', 
                                                'fontsize':'x-small'
                                                # title_fontsizeint or {'xx-small', 'x-small', 'small'
                                                } ,
                                aspect=1
                                        )
    
    vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
    cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite,  crs=crs , zoom=12) 
    cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels,   crs=crs, zoom=11) # zoom=13
    plt.title(f'{title}',fontsize=16)
    plt.xticks([])
    plt.yticks([]) 
    handles = [Line2D([0], [0], marker='s', color='w', markerfacecolor=k, label=v, markersize=8) for v, k in color_map.items()]
    ax.legend(title=legend_title, handles=handles, bbox_to_anchor=(1, 1),markerscale=1.01,title_fontsize='small',fontsize='x-small', loc='upper right')
    plt.tight_layout()
    # ax.figure.savefig(map_output)
    ax.figure.savefig(map_output,  bbox_inches='tight' ,   dpi = 400)

**OSM Network: Get Travel Time and Shortest path from a given point or an edge of a hotspot**

In [6]:
def get_network_stats(polygon):
     try:
          # polygon = cluster_boundaries.iloc[index]['geometry'] 
          graph = ox.graph.graph_from_polygon(polygon, network_type='drive')
          # Retrieve only edges from the graph
          edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)
          graph_proj = ox.project_graph(graph)
          # fig, ax = ox.plot_graph(graph_proj) 
          # Get Edges and Nodes
          nodes_proj, edges_proj = ox.graph_to_gdfs(graph_proj, nodes=True, edges=True)
          # print("Coordinate system:", edges_proj.crs)
          # edges_proj.head()
          # Get Edges and Nodes
          nodes_proj, edges_proj = ox.graph_to_gdfs(graph_proj, nodes=True, edges=True)
          # print("Network Coordinate system: projected", edges_proj.crs)
          # edges_proj.head()
          # Get the Convex Hull of the network
          convex_hull = edges_proj.unary_union.convex_hull
          # Calculate the area
          area = convex_hull.area
          # Calculate statistics with density information
          stats = ox.stats.basic_stats(graph_proj, area=area) #replace area by city polygon
          stats =pd.Series(stats) #edge_density_km(road density) - edge_length_total per sq km
     except:
          stats=0
          stats =pd.Series(stats) 
          pass     
     return stats

def get_osm_network():
    G = ox.graph_from_place(place, network_type='drive')
    # impute speed on all edges missing data
    G = ox.add_edge_speeds(G)
    # calculate travel time (seconds) for all edges
    G = ox.add_edge_travel_times(G)
    # see mean speed/time values by road type
    edges = ox.graph_to_gdfs(G, nodes=False)
    edges['highway'] = edges['highway'].astype(str)
    edges.groupby('highway')[['length', 'speed_kph', 'travel_time']].mean().round(1)

    # same thing again, but this time pass in a few default speed values (km/hour)
    # to fill in edges with missing `maxspeed` from OSM
    hwy_speeds = {'residential': 35,
                'secondary': 50,
                'tertiary': 60}
    G = ox.add_edge_speeds(G, hwy_speeds)
    G = ox.add_edge_travel_times(G)
    # calculate two routes by minimizing travel distance vs travel time
    orig = list(G)[1]
    dest = list(G)[-1]
    route1 = nx.shortest_path(G, orig, dest, weight='length')
    route2 = nx.shortest_path(G, orig, dest, weight='travel_time')

    # compare the two routes
    route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, 'length')))
    route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, 'length')))
    route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, 'travel_time')))
    route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, 'travel_time')))
    print('Route 1 is', route1_length, 'meters and takes', route1_time, 'seconds.')
    print('Route 2 is', route2_length, 'meters and takes', route2_time, 'seconds.')

    # pick route colors
    c1 = 'r' #length
    c2 = 'b' #travel_time
    rc1 = [c1] * (len(route1) - 1)
    rc2 = [c2] * (len(route2) - 1)
    rc = rc1 + rc2
    nc = [c1, c1, c2, c2]

    # plot the routes
    fig, ax = ox.plot_graph_routes(G, [route1, route2], route_color=rc, route_linewidth=6,
                                node_size=0, bgcolor='k')
    location_point = (33.299896, -111.831638)
    G2 = ox.graph_from_point(location_point, dist=400, truncate_by_edge=True)
    # impute speed on all edges missing data
    G2 = ox.add_edge_speeds(G2)
    # calculate travel time (seconds) for all edges
    G2 = ox.add_edge_travel_times(G2)
    origin = (33.301821, -111.829871)
    destination = (33.301402, -111.833108)
    origin_node = ox.distance.nearest_nodes(G2, origin[1], origin[0])
    destination_node = ox.distance.nearest_nodes(G2, destination[1], destination[0])
    route = ox.shortest_path(G2, origin_node, destination_node)
    # compare the two routes
    route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G2, route, 'length')))
    route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G2, route, 'travel_time')))
    print('Route 1 is', route1_length, 'meters and takes', route1_time, 'seconds.')
    fig, ax = ox.plot_graph_route(G2, route, route_color="c", node_size=0)

def get_services_infrastructure(center_point, dist):

     tags={'leisure':['park', 'nature_reserve', 'protected_area', 'garden']}
     park= ox.features.features_from_point(center_point=center_point, tags=tags, dist=dist)
     
     tags={'amenity':['college', 'dancing_school', 'driving_school', 'kindergarten', \
                    'language_school', 'library', 'surf_school', 'toy_library', 'research_institute', \
                         'training', 'music_school', 'school', 'university'],
                         'landuse': 'education'
                         }
     education= ox.features.features_from_point(center_point=center_point, tags=tags, dist=dist)
     
     tags={'landuse':['commercial', 'industrial', 'construction', 'retail']                   }
     industry= ox.features.features_from_point(center_point=center_point, tags=tags, dist=dist)
     
     # tags={'building': True}
     tags={'building': 'government'}
     building= ox.features.features_from_point(center_point=center_point, tags=tags, dist=dist)
     return park , education, industry, building

def get_services_infrastructure_inside_poly(polygon):
     try:
          tags={'leisure':['park', 'nature_reserve', 'protected_area', 'garden']}
          park=ox.features.features_from_polygon(polygon, tags)
     except:
          park=0
          pass
     try:
          tags={'amenity':['college', 'dancing_school', 'driving_school', 'kindergarten', \
                         'language_school', 'library', 'surf_school', 'toy_library', 'research_institute', \
                              'training', 'music_school', 'school', 'university'],
                              'landuse': 'education'
                              }
          education=ox.features.features_from_polygon(polygon, tags)
     except:
          education=0
          pass
     try:
          tags={'landuse':['commercial', 'industrial', 'construction', 'retail']                   }
          industry=ox.features.features_from_polygon(polygon, tags)
     except:
          industry=0
          pass
     try:
          tags={'building': 'government'}
          building=ox.features.features_from_polygon(polygon, tags)
     except:
          building=0
          pass
     # print("park , education, industry, building:", park , education, industry, building)
     return park , education, industry, building


def multi2single(gpdf):
    gpdf_singlepoly = gpdf[gpdf.geometry.type == 'Polygon']
    gpdf_multipoly = gpdf[gpdf.geometry.type == 'MultiPolygon']

    for i, row in gpdf_multipoly.iterrows():
        Series_geometries = pd.Series(row.geometry)
        df = pd.concat([gp.GeoDataFrame(row, crs=gpdf_multipoly.crs).T]*len(Series_geometries), ignore_index=True)
        df['geometry']  = Series_geometries
        gpdf_singlepoly = pd.concat([gpdf_singlepoly, df])

    gpdf_singlepoly.reset_index(inplace=True, drop=True)
    return gpdf_singlepoly


def create_ahc_knn_clusters(db  , raster_val , WGS84_meters):
    db =db.to_crs(WGS84_meters)
    db['area_sqm'] = db.geometry.area
    cluster_variables = [raster_val]
    db_scaled = robust_scale(db[cluster_variables])
    w = KNN.from_dataframe(db, k=4) #k for four nearest neighbors
    # Specify cluster model with spatial constraint
    model = AgglomerativeClustering(
        linkage="ward", connectivity=w.sparse, n_clusters=2
    )
    # Fit algorithm to the data
    model.fit(db_scaled)
    db["ward5wknn"] = model.labels_
    db['ward5wknn_sum'] = db[cluster_variables].groupby(db['ward5wknn']).transform('sum')
    db['area_sum'] = db['area_sqm'].groupby(db['ward5wknn']).transform('sum')
    db['area_sum'] = db['area_sum'].apply(lambda x: x/10000) #Hectares
    visualize_var= 'ward5wknn_sum_per_meter'
    db[visualize_var] = db['ward5wknn_sum'].div(db['area_sum']).round(2)
    # select the columns that you with to use for the dissolve and that will be retained
    cluster_boundaries = db[[raster_val , visualize_var,"ward5wknn", "geometry", "area_sqm", "area_sum", "ward5wknn_sum"]]
    # dissolve the state boundary by region 
    cluster_boundaries = cluster_boundaries.dissolve(by="ward5wknn" , aggfunc={raster_val: "sum", visualize_var: "sum" , "area_sqm":"sum" , "area_sum": "sum", "ward5wknn_sum": "sum",} , )
    cluster_boundaries=cluster_boundaries.explode().reset_index() #convert multipolygons to single part polygons. Multis cause errors
    cluster_boundaries["area_for_dupes"]=cluster_boundaries.geometry.area
#     cluster_boundaries=cluster_boundaries.sort_values('area_for_dupes', ascending=False).drop_duplicates(["ward5wknn"])
    cluster_boundaries=multi2single(cluster_boundaries)
    cluster_boundaries = cluster_boundaries.to_crs(crs)
    cluster_boundaries["centroid"] = cluster_boundaries["geometry"].centroid
    cluster_boundaries['lon'] = cluster_boundaries['centroid'].x
    cluster_boundaries['lat'] = cluster_boundaries['centroid'].y
    cluster_boundaries['cluster_area_deg'] = cluster_boundaries.geometry.area
    return db , cluster_boundaries, visualize_var


def create_ahc_knn_clusters_vito(db  , raster_val , WGS84_meters):
     db =db.to_crs(WGS84_meters)
     db['area_sqm'] = db.geometry.area
     cluster_variables = [raster_val]
     db_scaled = robust_scale(db[cluster_variables])
     w = KNN.from_dataframe(db, k=4) #k for four nearest neighbors
     # Specify cluster model with spatial constraint
     model = AgglomerativeClustering(
          linkage="ward", connectivity=w.sparse, n_clusters=3
     )
     # Fit algorithm to the data
     model.fit(db_scaled)
     db["ward5wknn"] = model.labels_
     db['ward5wknn_sum'] = db[cluster_variables].groupby(db['ward5wknn']).transform('sum')
     db['area_sum'] = db['area_sqm'].groupby(db['ward5wknn']).transform('sum')
     db['area_sum'] = db['area_sum'].apply(lambda x: x/10000) #Hectares
     visualize_var= 'ward5wknn_sum_per_meter'
     db[visualize_var] = db['ward5wknn_sum'].div(db['area_sum']).round(2)
     # select the columns that you with to use for the dissolve and that will be retained
     cluster_boundaries = db[[raster_val , visualize_var,"ward5wknn", "geometry", "area_sqm", "area_sum", "ward5wknn_sum"]]
     # dissolve the state boundary by region 
     cluster_boundaries = cluster_boundaries.dissolve(by="ward5wknn" , aggfunc={raster_val: "sum", visualize_var: "sum" , "area_sqm":"sum" , "area_sum": "sum", "ward5wknn_sum": "sum",} , )
     cluster_boundaries=cluster_boundaries.explode().reset_index() #convert multipolygons to single part polygons. Multis cause errors
     cluster_boundaries["area_for_dupes"]=cluster_boundaries.geometry.area
     # cluster_boundaries=cluster_boundaries.sort_values('area_for_dupes', ascending=False).drop_duplicates(["ward5wknn"])
     cluster_boundaries=multi2single(cluster_boundaries)
     cluster_boundaries = cluster_boundaries.to_crs(crs)
     cluster_boundaries["centroid"] = cluster_boundaries["geometry"].centroid
     cluster_boundaries['lon'] = cluster_boundaries['centroid'].x
     cluster_boundaries['lat'] = cluster_boundaries['centroid'].y
     cluster_boundaries['cluster_area_deg'] = cluster_boundaries.geometry.area
     return db , cluster_boundaries, visualize_var


def dissolve_categorical(db  , raster_val , WGS84_meters):
     db =db.to_crs(WGS84_meters)
     db['area_sqm'] = db.geometry.area
     cluster_variables = [raster_val]
     # db_scaled = robust_scale(db[cluster_variables])
     # w = KNN.from_dataframe(db, k=4) #k for four nearest neighbors
     # # Specify cluster model with spatial constraint
     # model = AgglomerativeClustering(
     #      linkage="ward", connectivity=w.sparse, n_clusters=15
     # )
     # # Fit algorithm to the data
     # model.fit(db_scaled)
     # db["ward5wknn"] = model.labels_
     db["ward5wknn"] = db[raster_val]
     db['ward5wknn_sum'] = db[cluster_variables].groupby(db['ward5wknn']).transform('sum')
     db['area_sum'] = db['area_sqm'].groupby(db['ward5wknn']).transform('sum')
     db['area_sum'] = db['area_sum'].apply(lambda x: x/10000) #Hectares
     visualize_var= 'ward5wknn_sum_per_meter'
     db[visualize_var] = db['ward5wknn_sum'].div(db['area_sum']).round(2)
     # select the columns that you with to use for the dissolve and that will be retained
     cluster_boundaries = db[[raster_val , visualize_var,"ward5wknn", "geometry", "area_sqm", "area_sum", "ward5wknn_sum"]]
     # dissolve the state boundary by region 
     cluster_boundaries = cluster_boundaries.dissolve(by="ward5wknn" , aggfunc={raster_val: "sum", visualize_var: "sum" , "area_sqm":"sum" , "area_sum": "sum", "ward5wknn_sum": "sum",} , )
     cluster_boundaries=cluster_boundaries.explode().reset_index() #convert multipolygons to single part polygons. Multis cause errors
     cluster_boundaries["area_for_dupes"]=cluster_boundaries.geometry.area
     # cluster_boundaries=cluster_boundaries.sort_values('area_for_dupes', ascending=False).drop_duplicates(["ward5wknn"])
     cluster_boundaries=multi2single(cluster_boundaries)
     cluster_boundaries = cluster_boundaries.to_crs(crs)
     cluster_boundaries["centroid"] = cluster_boundaries["geometry"].centroid
     cluster_boundaries['lon'] = cluster_boundaries['centroid'].x
     cluster_boundaries['lat'] = cluster_boundaries['centroid'].y
     cluster_boundaries['cluster_area_deg'] = cluster_boundaries.geometry.area
     return db , cluster_boundaries, visualize_var



def describe_cluster_trees_pop(cluster_boundaries, category, maps):
     df = pd.DataFrame(cluster_boundaries.drop(columns='geometry'))
     df = df.sort_values('cluster_area_deg', ascending = False).groupby('ward5wknn').head(2)

     df = df.reset_index() 
     df['Rank'] = df['ward5wknn_sum_per_meter'].rank(method='dense', ascending=False)
     df= df.sort_values(['Rank'], ascending=[True])
     cent= cluster_boundaries.dissolve().centroid 
     # White house 38.8977° N, 77.0365° W. centroid of the city.
     lat1 = float(cent.centroid.y)
     long1= float(cent.centroid.x)
     list_statements= []
     stats_dict= defaultdict(list)
     for index, row in df.iterrows():
          lat2 = row['lat'] # cent.centroid.y  #38.8893
          long2 =row['lon'] # cent.centroid.x  # -77.0506
          # print(f"lat1 {lat1} , long1 {long1}, lat2 {lat2}, long2 {long2}")
          points = calcNSEW(lat1, long1, lat2, long2)
          rank= int(row['Rank'])
          rank= ordinal(rank)
          value= row['ward5wknn_sum_per_meter']
          statement= f"A cluster that is located in the {points} of the city, is ranked {rank} and has the value of {value}"
          list_statements.append(statement)
          try:
               polygon = cluster_boundaries.iloc[index]['geometry']
               cluster_area_deg= cluster_boundaries.iloc[index]['cluster_area_deg']
               ward5wknn= cluster_boundaries.iloc[index]['ward5wknn']
               park , education, industry, building=get_services_infrastructure_inside_poly(polygon)
               stats_dict["cluster"].append(index)
               stats_dict["ward5wknn"].append(ward5wknn)
               stats_dict["points"].append(points)
               stats_dict["cluster_area_deg"].append(cluster_area_deg)
               stats_dict["rank"].append(rank)
               stats_dict["value"].append(value)
          except:
               pass
          try:
               stats_dict["park"].append(len(park))
          except:
               pass

          try:
               stats_dict["education"].append(len(education))
          except:
               pass

          try:
               stats_dict["industry"].append(len(industry))
          except:
               pass

          try:
               stats_dict["building"].append(len(building))
          except:
               pass

          try:
               park["index"]=index
               park["points"]=points
               park["rank"]=rank
               park["value"]=value
               park.to_csv(f"{tables}/{index}_{category}_park_df.csv")
          except:
               pass

          try:
               education["index"]=index
               education["points"]=points
               education["rank"]=rank
               education["value"]=value
               education.to_csv(f"{tables}/{index}_{category}_education_df.csv")
          except:
               pass

          try:
               industry["index"]=index
               industry["points"]=points
               industry["rank"]=rank
               industry["value"]=value
               industry.to_csv(f"{tables}/{index}_{category}_industry_df.csv")
          except:
               pass

          try:
               building["index"]=index
               building["points"]=points
               building["rank"]=rank
               building["value"]=value
               building.to_csv(f"{tables}/{index}_{category}_building_df.csv")               
          except:
               pass

          try:
               stats=get_network_stats(polygon)
               stats = stats.to_frame(0).T
               stats.to_csv(f"{tables}/{index}_{category}_infra_network_stats.csv")
          except:
               print(f"no network for {index}")
               pass

     statement_df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in stats_dict.items() ]))
     statement_df.to_csv(f"{tables}/{category}_amenities_stats_all.csv")
     return list_statements

# place = 'Nis, Serbia'
# get_osm_network_and_travel_time=get_osm_network()


In [7]:
def download_landsurface_temperature(city, cities_reprojected, max_value , years, month0 ,  month1):

    """
    download_landsurface_temperature: Exports Land Surface temperature rasters for a given country in the cities shapefile

    :param country: Name of the country
    :param cities_reprojected: geopandas read cities shapefile
    :param max_value: cap the number of cities
    :param years: years
    :param month0 : first hottest month
    :param month1 : last hottest month

    :return: confirms the export of rasters

    """

    # Create a jsondictionary from the geometry of the  shapefile
    geom = cities_reprojected['geometry']
    jsonDict = eval(geom.to_json())

    for index, row in cities_reprojected.iterrows():
        #   print(1)
        #   city= row['NAME_1'].lower() #instruct the column for city name in the city shapefile
          if index  <= max_value:
              city_number= index
              print(1)
              for x in jsonDict['features'][city_number]['geometry']['coordinates']:
                  try:
                      AOI = ee.Geometry.Polygon(x) #cast polygon as ee geometry
                      landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
                      #   print(3)
                      # Filter for hottest months in the past X years
                      def filter_hot_month(i):
                          return ee.Filter.date(years[i] + '-' + month0 + '-01', years[i] + '-' + month1 + '-01')

                      range_list= list(map(filter_hot_month, list(range(0, 10)))) #combination of months and years
                      rangefilter = ee.Filter.Or(range_list)
                    #   print(4)
                      # Define a function to scale the data and mask unwanted pixels
                      def maskL457sr(image):
                          # Bit 0 - Fill
                          # Bit 1 - Dilated Cloud
                          # Bit 2 - Cirrus (high confidence)
                          # Bit 3 - Cloud
                          # Bit 4 - Cloud Shadow
                          qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
                          saturationMask = image.select('QA_RADSAT').eq(0)
                          # Apply the scaling factors to the appropriate bands.
                          thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)
                          # Replace the original bands with the scaled ones and apply the masks.
                          return image.addBands(thermalBand, None, True).updateMask(qaMask).updateMask(saturationMask)
                      #   print(5)
                      # Apply filter and mask
                      collectionSummer = landsat.filter(rangefilter).filterBounds(AOI).map(maskL457sr).select('ST_B10').mean().add(-273.15).clip(AOI)
                      # collectionSummer=collectionSummer.toFloat()

                      lst_tif = os.path.join(rasters, 'lst_projected.tif')
                      geemap.ee_export_image( 
                                            # collectionSummer.toUint8(), 
                                            collectionSummer.toFloat(), 
                                            filename=lst_tif,  
                                            crs=EPSG_str,
                                            # crs_transform=crs_transform,
                                            scale=30, 
                                            region=AOI
                                            # file_per_band=False
                                            )
                        
                    #   print(6)
                  except Exception as e:
                      print(f"Error : {e} ")
                  return  lst_tif


In [8]:
%%capture
if __name__ ==  '__main__': 
    if run_tree_and_pop_analysis:
        # # City level map
        df_filtered= gdf_city
        title="City Boundary"
        legend_title= 'Boundary'
        visualize_column="shapeName"
        map_output= os.path.join(maps, f"{city}_{title}_map.png")
        export = map_city(maps=maps, vector_file=gdf_city, crs=crs, legend_title=legend_title,  \
                            visualize_column=visualize_column, title=title, map_output=map_output)

        crs = 4326
        fc_filtered  , region= create_fc_collection(gdf_city= gdf_city) 

        
        #-----------------------------------------------------------------------------------#

        # #Args
        cities_reprojected=gdf_city.reset_index()
        max_value=len(cities_reprojected)
        years =list(str(i) for i in range(2013, 2024)) #years from 2013-2023
        month0 = '06'  # first hottest month  # update for each country
        month1 = '09'  # end of hottest month (note that this is exclusive)  # update for each country
        country= 'SRB' # replace with the relevent country name
        lst_t= download_landsurface_temperature( city, cities_reprojected, max_value, years, month0, month1)
        raster_filename= "land_surface_temperature"
        unprojected_raster =lst_t  
        projected_raster = os.path.join(rasters, f'{raster_filename}.tif')
        reproject_rast= reproject_rasters(crs=crs, output_dir=rasters, unprojected_raster=unprojected_raster, \
                                projected_raster=projected_raster)

        output_shapefile = f'{shapefiles}/polygonized_{raster_filename}.shp'
        mask_value=None
        variable= raster_filename
        tif = projected_raster #os.path.join(rasters, 'tree_cover_projected.tif')
        raster_val= variable[0:9]
        name = polygonize(raster_file=projected_raster ,output_shapefile=output_shapefile ,crs=EPSG_str,  \
                            mask_value=mask_value, raster_val=raster_val)  

        cmap = "OrRd"
        # cmap = "Greens"
        # cmap = "YlGn"
        legends_format= '{:,.0f}'
        legend_title= "LST"
        map_title= f"Land Surface Temperature across {city}"
        scheme="quantiles"
        categorical=False
        map_output= f'{maps}/{city}_map_{raster_filename}.png'
        polygonized_shapefile = gpd.read_file(output_shapefile).to_crs(crs)
        newdf = overlay(polygonized_shapefile, gdf_city, how="intersection")

        export_the_map= map_func(maps= maps, vector_file=gdf_city , \
                hexagons=newdf,legend_title=legend_title, legends_format=legends_format,  \
                    crs=crs, cmap=cmap , visualize_column=raster_val,  \
                        title=map_title, map_output=map_output, scheme=scheme,  categorical=categorical)

        db , cluster_boundaries, visualize_var =create_ahc_knn_clusters(db = newdf , raster_val=raster_val, \
                                                                        WGS84_meters=WGS84_meters)
        map_title= f"Land Surface Temperature across {city} clusters"
        legends_format= '{:,.0f}'
        categorical=True
        map_output= f'{maps}/{city}_map_clusters_{raster_filename}.png'
        export_the_map= map_func(maps= maps, vector_file=gdf_city , \
            hexagons=db,legend_title=legend_title, legends_format=legends_format,  \
                crs=crs, cmap=cmap , visualize_column=visualize_var,  \
                    title=map_title, map_output=map_output,  scheme=scheme,  categorical=categorical)
        category=raster_filename
        describe_cluster_trees= describe_cluster_trees_pop(cluster_boundaries, category, maps)
        delete_shapefil= delete_shapefile(output_shapefile=output_shapefile)


        # legend_title= "Surface Temperature"
        # map_title= f"Land Surface Temperature across {city} cluster"
        # map=map_clusters(maps=maps, hexagons=cluster_boundaries, legend_title=legend_title, \
        #                 legends_format=legends_format, crs=crs ,cmap=cmap,  \
        #                     visualize_column=raster_val, title=map_title)
                        
        cluster_boundaries_lst= cluster_boundaries
        db_lst = db[[f"{raster_val}",'geometry' , 'ward5wknn']]

        #-----------------------------------------------------------------------------------#

        tree_cover_tif_esa= download_tree_cover_esa_vito(ee , EPSG_str, region)
        # raster_filename= "tree_cover_esa"
        raster_filename= "esa_tree_cover"
        unprojected_raster =tree_cover_tif_esa  
        projected_raster = os.path.join(rasters, f'{raster_filename}.tif')
        reproject_rast= reproject_rasters(crs=crs, output_dir=rasters, unprojected_raster=unprojected_raster, \
                                            projected_raster=projected_raster)

        output_shapefile = f'{shapefiles}/polygonized_{raster_filename}.shp'
        mask_value=None
        variable= raster_filename
        tif = projected_raster #os.path.join(rasters, 'tree_cover_projected.tif')
        raster_val= variable[0:9]
        name = polygonize(raster_file=projected_raster ,output_shapefile=output_shapefile ,crs=EPSG_str,  \
            mask_value=mask_value, raster_val=raster_val)  

        # cmap = "OrRd"
        cmap = "Greens"
        # cmap = "YlGn"
        legends_format= '{:,.0f}'
        legend_title= "Tree Cover"
        map_title= f"Tree Cover (ESA 10 meters) across {city}"
        scheme="quantiles"
        categorical=False
        map_output= f'{maps}/{city}_map_{raster_filename}.png'
        polygonized_shapefile = gpd.read_file(output_shapefile).to_crs(crs)
        newdf = overlay(polygonized_shapefile, gdf_city, how="intersection")

        export_the_map= map_func(maps= maps, vector_file=gdf_city , \
                hexagons=newdf,legend_title=legend_title, legends_format=legends_format,  \
                    crs=crs, cmap=cmap , visualize_column=raster_val,  \
                        title=map_title, map_output=map_output, scheme=scheme,  categorical=categorical)

        db , cluster_boundaries, visualize_var =create_ahc_knn_clusters(db = newdf, raster_val=raster_val, \
                                                                        WGS84_meters=WGS84_meters)
        map_title= f"Tree Cover (ESA 10 meters) across {city} clusters"
        legends_format= '{:,.0f}'
        categorical=True
        map_output= f'{maps}/{city}_map_clusters_{raster_filename}.png'
        export_the_map= map_func(maps= maps, vector_file=gdf_city , \
            hexagons=db,legend_title=legend_title, legends_format=legends_format,  \
                crs=crs, cmap=cmap , visualize_column=visualize_var,  \
                    title=map_title, map_output=map_output,  scheme=scheme,  categorical=categorical)
        category=raster_filename
        describe_cluster_trees= describe_cluster_trees_pop(cluster_boundaries, category, maps)
        delete_shapefil= delete_shapefile(output_shapefile=output_shapefile)


        # legend_title= "Tree Cover"
        # map_title= f"Tree Cover across {city} cluster"
        # map=map_clusters(maps=maps, hexagons=cluster_boundaries, legend_title=legend_title, \
        #                 legends_format=legends_format, crs=crs ,cmap=cmap,  \
        #                     visualize_column=raster_val, title=map_title)



        db_esa = db[[f"{raster_val}",'geometry' , 'ward5wknn']]


        #-----------------------------------------------------------------------------------#

        tree_cover_tif= download_tree_cover(ee , EPSG_str, region)
        raster_filename= "land_tree"
        unprojected_raster =tree_cover_tif  
        projected_raster = os.path.join(rasters, f'{raster_filename}.tif')
        reproject_rast= reproject_rasters(crs=crs, output_dir=rasters, unprojected_raster=unprojected_raster, \
                                          projected_raster=projected_raster)

        output_shapefile = f'{shapefiles}/polygonized_{raster_filename}.shp'
        mask_value=None
        variable= raster_filename
        tif = projected_raster#os.path.join(rasters, 'tree_cover_projected.tif')
        raster_val= variable[0:9]
        name = polygonize(raster_file=projected_raster ,output_shapefile=output_shapefile ,crs=EPSG_str,  \
            mask_value=mask_value, raster_val=raster_val)  

        # cmap = "OrRd"
        cmap = "Greens"
        # cmap = "YlGn"
        legends_format= '{:,.0f}'
        legend_title= "Tree Cover"
        map_title= f"Tree Cover (Landsat 30 meters) across {city}"
        scheme="quantiles"
        categorical=False
        map_output= f'{maps}/{city}_map_{raster_filename}.png'
        polygonized_shapefile = gpd.read_file(output_shapefile).to_crs(crs)
        newdf = overlay(polygonized_shapefile, gdf_city, how="intersection")

        export_the_map= map_func(maps= maps, vector_file=gdf_city , \
                hexagons=newdf,legend_title=legend_title, legends_format=legends_format,  \
                    crs=crs, cmap=cmap , visualize_column=raster_val,  \
                        title=map_title, map_output=map_output, scheme=scheme,  categorical=categorical)

        db , cluster_boundaries, visualize_var =create_ahc_knn_clusters(db = newdf , raster_val=raster_val, \
                                                                        WGS84_meters=WGS84_meters)
        map_title= f"Tree Cover (Landsat 30 meters) across {city} clusters"
        legends_format= '{:,.0f}'
        categorical=True
        map_output= f'{maps}/{city}_map_clusters_{raster_filename}.png'

        export_the_map= map_func(maps= maps, vector_file=gdf_city , \
            hexagons=db,legend_title=legend_title, legends_format=legends_format,  \
                crs=crs, cmap=cmap , visualize_column=visualize_var,  \
                    title=map_title, map_output=map_output,  scheme=scheme,  categorical=categorical)

        category=raster_filename
        describe_cluster_trees= describe_cluster_trees_pop(cluster_boundaries, category, maps)
        delete_shapefil= delete_shapefile(output_shapefile=output_shapefile)


        # legend_title= "Tree Cover (Landsat 30 meters)"
        # map_title= f"Tree Cover (Landsat 30 meters) across {city} cluster"
        # map=map_clusters(maps=maps, hexagons=cluster_boundaries, legend_title=legend_title, \
        #                 legends_format=legends_format, crs=crs ,cmap=cmap,  \
        #                     visualize_column=raster_val, title=map_title)

        db_landsat = db[[f"{raster_val}",'geometry' , 'ward5wknn']]

        #-----------------------------------------------------------------------------------#

        pop_density_tif= dowload_pop_density(ee , EPSG_str, region)
        raster_filename= "pop_density"
        unprojected_raster =pop_density_tif  
        projected_raster = os.path.join(rasters, f'{raster_filename}.tif')
        reproject_rast= reproject_rasters(crs=crs, output_dir=rasters, unprojected_raster=unprojected_raster, \
            projected_raster=projected_raster)

        output_shapefile = f'{shapefiles}/polygonized_{raster_filename}.shp'
        mask_value=None
        variable= raster_filename
        tif = projected_raster #os.path.join(rasters, 'tree_cover_projected.tif')
        raster_val= variable[0:9]
        name = polygonize(raster_file=projected_raster ,output_shapefile=output_shapefile ,crs=EPSG_str,  \
            mask_value=mask_value, raster_val=raster_val)  

        cmap = "OrRd"
        legends_format= '{:,.0f}'
        legend_title= "Population"
        map_title= f"Population (Worldpop 100 meters) across {city}"
        scheme="quantiles"
        categorical=False
        map_output= f'{maps}/{city}_map_{raster_filename}.png'
        polygonized_shapefile = gpd.read_file(output_shapefile).to_crs(crs)
        newdf = overlay(polygonized_shapefile, gdf_city, how="intersection")

        export_the_map= map_func(maps= maps, vector_file=gdf_city , \
                hexagons=newdf,legend_title=legend_title, legends_format=legends_format,  \
                    crs=crs, cmap=cmap , visualize_column=raster_val,  \
                        title=map_title, map_output=map_output, scheme=scheme,  categorical=categorical)

        db , cluster_boundaries, visualize_var =create_ahc_knn_clusters(db = newdf , raster_val=raster_val, \
                                                                        WGS84_meters=WGS84_meters)

        map_title= f"Population (Worldpop 100 meters) \n across {city} clusters"
        legends_format= '{:,.0f}'
        categorical=True
        map_output= f'{maps}/{city}_map_clusters_{raster_filename}.png'
        export_the_map= map_func(maps= maps, vector_file=gdf_city , \
            hexagons=db,legend_title=legend_title, legends_format=legends_format,  \
                crs=crs, cmap=cmap , visualize_column=visualize_var,  \
                    title=map_title, map_output=map_output,  scheme=scheme,  categorical=categorical)

        category=raster_filename
        describe_cluster_trees= describe_cluster_trees_pop(cluster_boundaries, category, maps)
        delete_shapefil= delete_shapefile(output_shapefile=output_shapefile)

        # legend_title= "Population"
        # map_title= f"Population (Worldpop 100 meters) across {city} cluster"
        # map=map_clusters(maps=maps, hexagons=cluster_boundaries, legend_title=legend_title, \
        #                 legends_format=legends_format, crs=crs ,cmap=cmap,  \
        #                     visualize_column=raster_val, title=map_title)


        db_pop = db[[f"{raster_val}",'geometry' , 'ward5wknn']]


        #-----------------------------------------------------------------------------------#

        # google_dynamic_worldcover= download_google_dynamic_worldcover(ee , EPSG_str, region)
        esri_dynamic_worldcover= download_esri_dynamic_worldcover(ee , EPSG_str, region)

        # Esri cover
        raster_filename= "esri_dynamic_worldcover"
        unprojected_raster =esri_dynamic_worldcover  
        projected_raster = os.path.join(rasters, f'{raster_filename}.tif')
        reproject_rast= reproject_rasters(crs=crs, output_dir=rasters, unprojected_raster=unprojected_raster, \
            projected_raster=projected_raster)

        output_shapefile = f'{shapefiles}/polygonized_{raster_filename}.shp'
        mask_value=None
        variable= raster_filename
        tif = projected_raster #os.path.join(rasters, 'tree_cover_projected.tif')
        raster_val= variable[0:9]
        name = polygonize(raster_file=projected_raster ,output_shapefile=output_shapefile ,crs=EPSG_str,  \
                            mask_value=mask_value, raster_val=raster_val)  

        cmap = "OrRd"
        legends_format= '{:,.0f}'
        legend_title= "Land Cover"
        map_title= f"Land Cover (ESRI 10 meters) across {city}"
        scheme="quantiles"
        categorical=False
        map_output= f'{maps}/{city}_map_{raster_filename}.png'
        polygonized_shapefile = gpd.read_file(output_shapefile).to_crs(crs)
        newdf = overlay(polygonized_shapefile, gdf_city, how="intersection")
        map_output= f'{maps}/{city}_map_{raster_filename}.png'
        
        export_the_map= map_land_cover(vector_file=gdf_city , \
                                        hexagons=newdf,title=map_title, legend_title=legend_title, \
                                            crs=crs,  visualize_column=raster_val, map_output=map_output)


        db , cluster_boundaries, visualize_var =dissolve_categorical(db = newdf , raster_val=raster_val, \
                                                                        WGS84_meters=WGS84_meters)
        map_title= f"Land Cover (ESRI 10 meters) across {city} clusters"
        legends_format= '{:,.0f}'
        categorical=True
        map_output= f'{maps}/{city}_map_clusters_{raster_filename}.png'
        export_the_map= map_land_cover(vector_file=gdf_city , \
                                    hexagons=db,title=map_title, legend_title=legend_title, \
                                        crs=crs,  visualize_column=raster_val, map_output=map_output)

        category=raster_filename
        describe_cluster_trees= describe_cluster_trees_pop(cluster_boundaries, category, maps)
        delete_shapefil= delete_shapefile(output_shapefile=output_shapefile)

        # legend_title= "Land Cover"
        # map_title= f"Land Cover (ESRI 10 meters) across {city} cluster"
        # map=map_clusters(maps=maps, hexagons=cluster_boundaries, legend_title=legend_title, \
        #                 legends_format=legends_format, crs=crs ,cmap=cmap,  \
        #                     visualize_column=raster_val, title=map_title)


        db_lc = db[[f"{raster_val}",'geometry' , 'ward5wknn']]

        #-----------------------------------------------------------------------------------#
        # Put everything togther
        #-----------------------------------------------------------------------------------#

        # cluster_boundaries_lst=cluster_boundaries_lst.to_crs(crs)
        # db_lc=db_lc.to_crs(crs)
        # db_lc['geometry'] = db_lc['geometry'].centroid
        # df = gpd.sjoin(cluster_boundaries_lst, db_lc, how='left', op='contains')
        # db_pop= db_pop.to_crs(crs)
        # df = gpd.sjoin(df, db_pop, how='left', op='contains')
        # db_landsat= db_landsat.to_crs(crs)
        # df = gpd.sjoin(df, db_landsat, how='left', op='contains')
        # db_esa= db_esa.to_crs(crs)
        # df = gpd.sjoin(df, db_esa, how='left', op='contains')

        # # # # Place all other dbs i  heat and then do some regs and descriptive stats
        # # cluster_boundaries_lst=cluster_boundaries_lst.to_crs(crs)
        # # raster_filename= "land_surface_temperature"
        # # db_var=db_var.to_crs(crs)
        # # db_var['geometry'] = db_var['geometry'].centroid
        # # df_all= overlay(cluster_boundaries_lst, db_var, how="intersection" , keep_geom_type=False)
        # # df_all=df_all.sort_values('area_for_dupes', ascending=False).drop_duplicates(["geometry"])
        # # df_all
        # #-----------------------------------------------------------#
        # #-----------------------------------------------------------#
        # # Group table by cluster label, keep the variables used 
        # # for clustering, and obtain their descriptive summary
        # cluster_variables=[]
        # k5desc = db.groupby('ward5wknn')[cluster_variables].describe()
        # # Loop over each cluster and print a table with descriptives
        # for cluster in k5desc.T:
        #     print('\n\t---------\n\tCluster %i'%cluster)
        #     print(k5desc.T[cluster])
            



<a id='section3'></a>
<h5><center> <font color='cyan'> Section 3: Visualize Vito heat rasters at city level</font>   </center></h5>

In [9]:

def create_hexbins_and_geom_features(gdf_city, hexbin_res, WGS84):
    poly = gdf_city.geometry.unary_union
    gdf_boundary = gpd.GeoDataFrame(geometry=[poly],crs=gdf_city.crs)
    # gdf_h3 = gdf_boundary.h3.polyfill(9,explode=True)
    gdf_h3 = gdf_boundary.h3.polyfill(hexbin_res,explode=True)
    gdf_h3 = gdf_h3.set_index('h3_polyfill').h3.h3_to_geo_boundary()
    output_shapefile= f'{shapefiles}/h3_grid.shp'
    gdf_h3.to_file(output_shapefile)
    hexbins_projected = gpd.read_file(output_shapefile).to_crs(WGS84)

    geom = gdf_boundary['geometry']
    jsonDict = eval(geom.to_json())
    for index, row in gdf_boundary.iterrows(): 
        polygon_list= []
        for x in jsonDict['features'][index]['geometry']['coordinates']:
            polygon_list.append(x)
            region = ee.Geometry.Polygon(polygon_list)
            fc_filtered = ee.FeatureCollection(region)  
    # gdf_h3.plot()
    return fc_filtered  , region , hexbins_projected

def clip_and_export_raster(input_gpdf, raster , out_raster, zonal_stat_var, hexbins_projected):
    Vector=input_gpdf
    with rasterio.open(raster) as src:
        Vector=Vector.to_crs(src.crs)
        # print(f'Gdf srs: {Vector.crs}')
        out_image, out_transform=mask(src,Vector.geometry,crop=True)
        out_meta=src.meta.copy() # copy the metadata of the source DEM
        
    out_meta.update({
        "driver":"Gtiff",
        "height":out_image.shape[1], # height starts with shape[1]
        "width":out_image.shape[2], # width starts with shape[2]
        "transform":out_transform
    })
                
    with rasterio.open(out_raster,'w',**out_meta) as dst:
        dst.write(out_image)

    # add zonal stats to the hexbin gpd dataframe and replace nans with 0 
    hexbins_projected[zonal_stat_var] = zonal_stats(hexbins_projected, out_raster ,stats='mean')
    hexbins_projected[zonal_stat_var] = [item['mean'] for item in hexbins_projected[zonal_stat_var]]
    hexbins_projected[zonal_stat_var].fillna(0,inplace=True)
    max=int(hexbins_projected.loc[hexbins_projected[zonal_stat_var].idxmax()][zonal_stat_var])
    
    return hexbins_projected , max

def get_uh_raster_dirs(input_raster_dir):
    raster_dirs=  [x[1] for x in os.walk(input_raster_dir)] # ['Present', 'SSP119' , 'SSP370']
    raster_dirs = [x for x in raster_dirs if len(x)>0]
    raster_dirs = list(np.concatenate(raster_dirs))
    return raster_dirs


def process_vito( input_raster_dir, gdf_city , crs):
    fc_filtered  , region , hexbins_projected= create_hexbins_and_geom_features(gdf_city, hexbin_res=8, WGS84=WGS84)
    extension = '.tif'
    for root, dirs_list, files_list in os.walk( input_raster_dir):
        print(root)
        for file_name in files_list:
            print(file_name)
            if os.path.splitext(file_name)[-1] == extension:
                file_name_path = os.path.join(root, file_name)
                file_name = os.path.splitext(file_name)[0]
                # file_name = ' '.join(file_name.split("_")[:-2])
                file_name = ' '.join(file_name.split("_")[:-2])
                out_rst = os.path.join(rasters, f'{file_name}.tif')
                hexbins_projected , max= clip_and_export_raster(input_gpdf=gdf_city, raster=file_name_path , out_raster=out_rst, \
                                                                    zonal_stat_var=file_name , hexbins_projected=hexbins_projected)
                cmap = "coolwarm"
                # cmap = "YlGn"
                legends_format= '{:,.02f}'
                legend_title= f"{file_name}"
                map_title= f"{file_name} \n across {city}"
                scheme="quantiles"
                categorical=False
                map_output= f'{maps}/map_{file_name}.png'
                
                export_the_map= map_func(maps= maps, vector_file=gdf_city , \
                        hexagons=hexbins_projected,legend_title=legend_title, legends_format=legends_format,  \
                            crs=crs, cmap=cmap , visualize_column=file_name,  \
                                title=map_title, map_output=map_output, scheme=scheme,  categorical=categorical)


                # export_the_map= map_func(maps= maps, vector_file=gdf_city , \
                #         hexagons=hexbins_projected,legend_title=legend_title, legends_format=legends_format,  \
                #             crs=crs, cmap=cmap , visualize_column=file_name,  \
                #                 title=map_title, map_output=map_output, scheme=scheme,  categorical=categorical)

                db , cluster_boundaries, visualize_var =create_ahc_knn_clusters_vito(db = hexbins_projected , \
                                                                                raster_val=file_name, WGS84_meters=WGS84_meters)
                map_title= f"{file_name} \n across {city} clusters"
                legends_format= '{:,.02f}'
                categorical=True
                map_output= f'{maps}/map_clusters_{file_name}.png'
                export_the_map= map_func(maps= maps, vector_file=gdf_city , \
                    hexagons=db,legend_title=legend_title, legends_format=legends_format,  \
                        crs=crs, cmap=cmap , visualize_column=visualize_var,  \
                            title=map_title, map_output=map_output,  scheme=scheme,  categorical=categorical)
                
                category=file_name
                describe_cluster_trees= describe_cluster_trees_pop(cluster_boundaries, category, maps)
                # delete_shapefil= delete_shapefile(output_shapefile=output_shapefile)
                # legend_title= "Population"
                # map_title= f"Population (Worldpop 100 meters) across {city} cluster"
                # map=map_clusters(maps=maps, hexagons=cluster_boundaries, legend_title=legend_title, \
                #                 legends_format=legends_format, crs=crs ,cmap=cmap,  \
                #                     visualize_column=raster_val, title=map_title)

                db_var= f"db_{file_name}"
                db_var = db[[f"{file_name}",'geometry' , 'ward5wknn']]




In [10]:
%%capture
if __name__ ==  '__main__': 
    if run_vito_analysis:
        # input_raster_dir = f'{base_dir}/data/Phnom Penh/Geotiffs'
        input_raster_dir = data
        # raster_dirs=get_uh_raster_dirs(input_raster_dir)
        process_vito( input_raster_dir=data, gdf_city=gdf_city , crs=crs)
        

<a id='section4'></a>
<h5><center> <font color='cyan'> Section 4: Creat the Heat Scan PPT</font>   </center></h5>


In [11]:
def add_city_map_slide(img_path ,tables, prs, width, height, var):
    prs.slide_height = Inches(height)
    prs.slide_width  = Inches(width+4)
    blank_slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(blank_slide_layout)
    left = Inches(4)
    top =Inches(0)
    width = Inches(width)
    height = Inches(height)
    pic = slide.shapes.add_picture(img_path, left, top, width = width, height = height)

    # Add key message box.
    # df_= points

    text_width = Inches(4)
    text_height = Inches(4)
    left =Inches(0)
    top =Inches(1)
    txBox = slide.shapes.add_textbox(left, top, text_width, text_height)
    tf = txBox.text_frame
    tf.word_wrap = True
    p = tf.add_paragraph()
    run = p.add_run()
    run.text = f'Key Message: {var}'
    font = run.font
    font.name = 'Yu Gothic'
    font.size = Pt(12)
    font.bold = True
    run.font.color.rgb = RGBColor(0, 0, 0)
    prs.save(f'{tables}/Heat_Scan.pptx')

def add_cluster_map_slide(img_path ,tables, prs, width, height, var):
    prs.slide_height = Inches(height)
    prs.slide_width  = Inches(width+4)
    blank_slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(blank_slide_layout)
    left = Inches(4)
    top =Inches(0)
    width = Inches(width)
    height = Inches(height)
    pic = slide.shapes.add_picture(img_path, left, top, width = width, height = height)

    # Add key message box.
    text_width = Inches(4)
    text_height = Inches(2)
    left =Inches(0)
    top =Inches(1)
    txBox = slide.shapes.add_textbox(left, top, text_width, text_height)
    tf = txBox.text_frame
    tf.word_wrap = True
    p = tf.add_paragraph()
    run = p.add_run()
    run.text = f'Key Message: For {var}, there are five clusters in the city.'
    font = run.font
    font.name = 'Yu Gothic'
    font.size = Pt(13)
    font.bold = True
    run.font.color.rgb = RGBColor(0, 0, 0)
    for cluster in range(0, 3):
        try:
            # print("read cluster")
            clusters_level=pd.read_csv(f"{tables}/{var}_amenities_stats_all.csv") 
            # print("read cluster",clusters_level)
            clusters_level= len(clusters_level)
        except:
            clusters_level=0
            pass

        try:
            parks= pd.read_csv(f"{tables}/{cluster}_{var}_park_df.csv") 
            parks= len(parks)
        except:
            parks=0
            pass
        try:
            education= pd.read_csv(f"{tables}/{cluster}_{var}_education_df.csv") 
            education= len(education)
        except:
            education=0
            pass

        try:
            industry= pd.read_csv(f"{tables}/{cluster}_{var}_industry_df.csv") 
            industry= len(industry)
        except:
            industry=0
            pass

        try:
            buildings= pd.read_csv(f"{tables}/{cluster}_{var}_building_df.csv") 
            buildings= len(buildings)
        except:
            buildings=0
            pass
        try:
            road_netwrok= pd.read_csv(f"{tables}/{cluster}_{var}_infra_network_stats.csv") 
            road_netwrok= len(road_netwrok)
        except:
            road_netwrok=0
            pass

        # Add key message box.
        text_width = Inches(4)
        text_height = Inches(4)
        left =Inches(0)
        top =Inches(3+cluster)
        txBox = slide.shapes.add_textbox(left, top, text_width, text_height)
        tf = txBox.text_frame
        tf.word_wrap = True
        p = tf.add_paragraph()
        run = p.add_run()
        run.text = f'Cluster {cluster} is ranked add rank and has {parks} parks, {education} education, {industry} industry, {buildings} buildings, {road_netwrok} road_network.'
        font = run.font
        font.name = 'Yu Gothic'
        font.size = Pt(12)
        font.bold = True
        run.font.color.rgb = RGBColor(0, 0, 0)
        
    prs.save(f'{tables}/Heat_Scan.pptx')


In [12]:
# cluster=1
# road_netwrok= pd.read_csv(f"{tables}/{cluster}_{var}_infra_network_stats.csv") 


In [13]:
%%capture
if __name__ ==  '__main__': 
    if create_the_ppt:
        prs = Presentation() #Create a new presentation. Add the map 
        var_list = glob.glob(os.path.join( maps, f"map_*.png"))
        for var_name in var_list:
            if "clusters" not in var_name:
                var_name= os.path.basename(var_name)
                # print(var_name)
                var_name=var_name.replace('map_','')
                var_name=var_name.replace('.png','')
                print(var_name)
                var=  var_name #"CSDI ssp370 2021 2040"    
                file_list = glob.glob(os.path.join( maps, f"map_{var}*.png"))
                for file_path in file_list:
                    file_name= os.path.basename(file_path)
                    add_city_map_slide( img_path=file_path ,tables=tables, prs=prs, width=width, height=height, var=var)
            elif "clusters" in var_name:
                var_name= os.path.basename(var_name)
                # print(var_name)
                var_name=var_name.replace('map_clusters_','')
                var_name=var_name.replace('.png','')
                print(var_name)
                var=  var_name #"CSDI ssp370 2021 2040"    
                file_list = glob.glob(os.path.join( maps, f"map_clusters_{var}*.png"))
                for file_path in file_list:
                    file_name= os.path.basename(file_path)
                    add_cluster_map_slide( img_path=file_path ,tables=tables, prs=prs, width=width, height=height, var=var)

                # # var=  "map_clusters_CSDI ssp370 2021 2040"   
                # # var=  "heatwave days present 2001 2020"   
                # file_list = glob.glob(os.path.join( maps, f"map_clusters_{var}*.png"))
                # for file_path in file_list:
                #     file_name= os.path.basename(file_path)
                #     add_cluster_map_slide( img_path=file_path ,tables=tables, prs=prs, width=width, height=height, var=var)



<a id='section5'></a>
<h5><center> <font color='cyan'> Section 5: Population density and tree cover Sub-city level</font>   </center></h5>

**Playground**